In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.optim import Adam
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, Lambda
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [2]:
def MNIST_loaders(train_batch_size=50000, test_batch_size=10000):
    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,)),
        Lambda(lambda x: torch.flatten(x))])

    train_loader = DataLoader(
        MNIST('./ignore_dir/data/', train=True,
              download=True,
              transform=transform),
        batch_size=train_batch_size, shuffle=True)

    test_loader = DataLoader(
        MNIST('./ignore_dir/data/', train=False,
              download=True,
              transform=transform),
        batch_size=test_batch_size, shuffle=False)

    return train_loader, test_loader


def overlay_y_on_x(x, y):
    x_ = x.clone()
    x_[:, :10] *= 0.0
    x_[range(x.shape[0]), y] = x.max()
    return x_


class Net(torch.nn.Module):
    def __init__(self, dims):
        super().__init__()
        self.layers = []
        for d in range(len(dims) - 1):
            self.layers += [Layer(dims[d], dims[d + 1]).cuda()]

    def predict(self, x):
        goodness_per_label = []
        for label in range(10):
            h = overlay_y_on_x(x, label)
            goodness = []
            for layer in self.layers:
                h = layer(h)
                goodness += [h.pow(2).mean(1)]
            goodness_per_label += [sum(goodness).unsqueeze(1)]
        goodness_per_label = torch.cat(goodness_per_label, 1)
        return goodness_per_label.argmax(1)

    def train(self, x_pos, x_neg):
        h_pos, h_neg = x_pos, x_neg
        for i, layer in enumerate(self.layers):
            print('training layer', i, '...')
            h_pos, h_neg = layer.train(h_pos, h_neg)


class Layer(nn.Linear):
    def __init__(self, in_features, out_features,
                 bias=True, device=None, dtype=None):
        super().__init__(in_features, out_features, bias, device, dtype)
        self.relu = torch.nn.ReLU()
        self.opt = Adam(self.parameters(), lr=0.03)
        self.threshold = 2.0
        self.num_epochs = 1000

    def forward(self, x):
        x_direction = x / (x.norm(2, 1, keepdim=True) + 1e-4)
        return self.relu(
            torch.mm(x_direction, self.weight.T) +
            self.bias.unsqueeze(0))

    def train(self, x_pos, x_neg):
        for i in tqdm(range(self.num_epochs)):
            g_pos = self.forward(x_pos).pow(2).mean(1)
            g_neg = self.forward(x_neg).pow(2).mean(1)
            loss = torch.log(1 + torch.exp(torch.cat([
                -g_pos + self.threshold,
                g_neg - self.threshold]))).mean()
            self.opt.zero_grad()
            loss.backward()
            self.opt.step()
        return self.forward(x_pos).detach(), self.forward(x_neg).detach()


In [4]:
torch.manual_seed(1234)
train_loader, test_loader = MNIST_loaders()

net = Net([784, 500, 500])
x, y = next(iter(train_loader))
x, y = x.cuda(), y.cuda()
x.size(0), y.size(0)

(50000, 50000)

In [8]:
rnd = torch.randperm(x.size(0))
rnd

sum(y == y[rnd])

tensor(5138, device='cuda:0')

In [ ]:
x_pos = overlay_y_on_x(x, y)
rnd = torch.randperm(x.size(0))
x_neg = overlay_y_on_x(x, y[rnd])

In [4]:
net.train(x_pos, x_neg)

print('train error:', 1.0 - net.predict(x).eq(y).float().mean().item())

x_te, y_te = next(iter(test_loader))
x_te, y_te = x_te.cuda(), y_te.cuda()

print('test error:', 1.0 - net.predict(x_te).eq(y_te).float().mean().item())

training layer 0 ...


100%|██████████| 1000/1000 [00:26<00:00, 37.37it/s]


training layer 1 ...


100%|██████████| 1000/1000 [00:21<00:00, 47.48it/s]


train error: 0.06754004955291748
test error: 0.06850004196166992


In [5]:
# train_loader, test_loader = MNIST_loaders()
# # x, y = next(iter(train_loader))
# # print(x.shape, y.shape)
#
# def overlay_y_on_x(x, y):
#     x_ = x.clone()
#
#     x_[:, :10] *= 0.0
#     x_[range(x.shape[0]), y] = x.max()
#
#     return x_
#
# xb = overlay_y_on_x(x, y)
# plt.imshow(xb.reshape((50000, 1, 28, 28)).numpy()[0].transpose((1, 2, 0)), cmap="gray")
# plt.show()

# index = 20
# xb, yb = x[index].unsqueeze(0), y[index].unsqueeze(0)
# xb = overlay_y_on_x(xb, yb)
# plt.imshow(xb.reshape((5, 1, 28, 28)).numpy()[0].transpose((1, 2, 0)), cmap="gray")
# plt.show()

SyntaxError: unmatched '}' (3937186872.py, line 1)